# This Notebook Trains MCCD.

In [1]:
ls data/bench/a

ls: data/bench/a: No such file or directory


In [5]:
!python train_1q_modules_cleaned.py \
        --code_distance 3 \
        --logical_circuit_index 3 \
        --if_final_round_syndrome \
        --batch_size 1024 \
        --run_index 0 \
        --model_save_path c3_d3 \
        --train_data_dir cached_qec_data_small/train \
        --val_data_dir cached_qec_data_small/val \
        --train_depth_list 1 2 3 4 5 \
        --val_depth_list 1 2 3 4 5 6 7 8 9 10


Namespace(code_distance=3, logical_circuit_index='3', batch_size=1024, if_final_round_syndrome=True, if_large_lstm_2q=False, load_model=None, run_index='0', model_save_path='c3_d3', train_data_dir='cached_qec_data_small/train', val_data_dir='cached_qec_data_small/val', train_depth_list=[1, 2, 3, 4, 5], val_depth_list=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], save_model_every_n_batches=500, validate_every_n_batches=100)
Number of params in model: 270240
Model save to: c3_d3
Save to trained_models/c3_d3
0it [00:00, ?it/s][Step 1024] Train Acc: 84.52% | Val Acc: 68.20%
Model checkpoint saved.
100it [00:04, 24.83it/s]
Final model saved.


In [6]:
!python train_2q_module_cleaned.py \
        --code_distance 3 \
        --logical_circuit_index 4 \
        --if_final_round_syndrome \
        --batch_size 1024 \
        --run_index 0 \
        --model_save_path c3_d4 \
        --train_data_dir cached_qec_data_small/train \
        --val_data_dir cached_qec_data_small/val \
        --train_depth_list 1 2 3 4 5 \
        --val_depth_list 1 2 3 4 5 6 7 8 9 10 \
        --load_model trained_models/c3_d3


Namespace(code_distance=3, logical_circuit_index='4', batch_size=1024, if_final_round_syndrome=True, if_large_lstm_2q=False, load_model='trained_models/c3_d3', run_index='0', model_save_path='c3_d4', train_data_dir='cached_qec_data_small/train', val_data_dir='cached_qec_data_small/val', train_depth_list=[1, 2, 3, 4, 5], val_depth_list=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], save_model_every_n_batches=500, validate_every_n_batches=100)
Loading model from: trained_models/c3_d3
Number of params in model: 108544
Model save to: c3_d4
Save to trained_models/c3_d4
[Step 1024] Train Acc: 76.07% | Val Acc: 56.50%
Model checkpoint saved.
Final model saved.
